# Pods

Podman has the hability to run containers in Pods, like in Kubernetes.

Pod is a concept in which a group of one or more containers working together for a common purpose and sharing the same namespaces and cgroups (resource constraints).
This way it is also possible to group containers's network space and volume mounts, making it easier for the containers to interact and work as if they were a single unit.


## Running Pods

To run containers in pods requires to plan a little bit, as the pod needs to be created up-front, specifying the volumes and ports that will be required.
Afterwards the containers will be attached to this pod.

We will run the following example to demonstrate how it works, start a simple http server in one pod, while in another we will refresh the index.html using a volume: 2 concerns (2 containers and 1 pod):

1. Create a pod
1. Create the web server container and attach it to the pod
1. Create the sidecar container that will update the index.html



### Create the Pod

After creating the path for the volume mount we will:

1. Specify the port binding (8080:8080)
1. Specify a volume mount to `/var/www/html` with the `z` parameter. `z` specifies that the volume mount should happen in each container (shared).

In [2]:
mkdir -p tests/html

: 1

In [3]:
podman pod create -p 8080:8080 --name dynamic-webserver --volume $PWD/tests/html:/var/www/html:z

0f2a20865fa08680675e1dd252d8efbd04c3d2d39483be8da16a4c2a834b0708


: 1

In [4]:
podman pod ps

POD ID        NAME               STATUS      CREATED        INFRA ID      # OF CONTAINERS
0f2a20865fa0  dynamic-webserver  Created     8 minutes ago  f7c1ba69e73e  1


: 1

### Attach the containers